# CS295/395: Secure Distributed Computation
## In-Class Exercise, 9/28/2020

## References

- Overview of the BGW protocol: [Pragmatic MPC, Section 3.3](https://securecomputation.org/docs/pragmaticmpc.pdf)
- Vandermonde Matrices for polynomial evaluation: [Asharov & Lindell, 2011, Section 3.3, Definition 3.6](https://eprint.iacr.org/2011/136.pdf)
- Formal protocol description (GRR protocol): [Lindell & Nof, 2017, Appendix B.3 (Protocol B.3)](https://eprint.iacr.org/2017/816.pdf)

In [ ]:
# Imports and definitions
import numpy as np
from collections import defaultdict
import urllib.request

_PRIME = 2 ** 13 - 1

shamir_lib_url = "https://raw.githubusercontent.com/jnear/cs295-secure-computation/master/utils/shamir.py"

### DANGER: this line is dangerous. Make sure the URL above is correct, and has correct code.
exec(urllib.request.urlopen(shamir_lib_url).read())

def share_shamir(t, n, x, prime=_PRIME):
    shares_with_x = share_input(x, minimum=t, shares=n, prime=prime)
    return [y for x,y in shares_with_x]

def reconstruct_shamir(shares, prime=_PRIME):
    shares_with_x = list(zip(range(1, len(shares)+1), shares))
    return recover_secret(shares_with_x, prime=prime)

class Party:
    """A participant in a multiparty computation protocol."""
    def __init__(self, field_size=_PRIME):
        """Initialize the field size and dictionary to hold received messages."""
        self.field_size = field_size
        self.input = None
        self.output = None
        self.received = defaultdict(list)
    
    def send(self, other, round, msg):
        """Simulate sending a message `msg` to another party `other` during round `round`"""
        other.received[round].append(msg)

    def get_view(self):
        """Returns the view of this party: its input, output, and received messages."""
        return (self.input, self.output, dict(self.received))

In [ ]:
# Finite field matrix inversion, algorithm from here:
# https://stackoverflow.com/questions/4287721/easiest-way-to-perform-modular-matrix-inversion-with-python

def inversemodp(a, p):
    a = a % p
    if (a == 0):
        #print "a is 0 mod p"
        return None
    if a > 1 and p % a == 0:
        return None
    #(x,y) = generalizedEuclidianAlgorithm(p, a % p);
    (x,y) = _extended_gcd(p, a % p);
    inv = y % p
    assert (inv * a) % p == 1
    return inv

def identitymatrix(n):
    return [[int(x == y) for x in range(0, n)] for y in range(0, n)]

def inversematrix(matrix, q):
    n = len(matrix)
    A = np.matrix([[ matrix[j, i] for i in range(0,n)] for j in range(0, n)], dtype = int)
    Ainv = np.matrix(identitymatrix(n), dtype = int)
    for i in range(0, n):
        factor = inversemodp(A[i,i], q)
        if factor is None:
             raise ValueError("Error: no factor")
        A[i] = A[i] * factor % q
        Ainv[i] = Ainv[i] * factor % q
        for j in range(0, n):
            if (i != j):
                factor = A[j, i]
                A[j] = (A[j] - factor * A[i]) % q
                Ainv[j] = (Ainv[j] - factor * Ainv[i]) % q
    return Ainv

## Question 1

Describe a protocol to multiply two input numbers. The input numbers will be secret-shared according to a $(t,n)$ Shamir secret sharing scheme before the protocol starts, and each party will receive one share of both numbers. Each party should output *one share of the product*, using a $(t, n)$ Shamir secret sharing scheme (i.e. the threshold for the output should be the same as the threshold for the input).

\begin{equation*}
\textbf{Functionality: Multiply Two Numbers}\\
\fbox{$\mathcal{F}(a, b) = a \cdot b$}
\end{equation*}




YOUR ANSWER HERE

## Question 2

Implement your protocol from question 1.

In [ ]:
class MultTwoParty(Party):
    def round1(self, parties, a_shr, b_shr, t):
        self.input = (a_shr, b_shr)
        self.parties = parties
        n = len(parties)
        assert t <= n/2
        
        # YOUR CODE HERE
        raise NotImplementedError()

    def round2(self):
        n = len(self.parties)
        
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
NUM_PARTIES = 6
# (t, n)-Shamir scheme
n = NUM_PARTIES
t = 3

shares1 = share_shamir(t, n, 5)
shares2 = share_shamir(t, n, 6)

parties = [MultTwoParty() for _ in range(NUM_PARTIES)]

for p,s1,s2 in zip(parties, shares1, shares2):
    p.round1(parties, s1, s2, t)
for p in parties:
    p.round2()
for p in parties:
    print(p.get_view())

output_shares = [p.output for p in parties]
print('Reconstruction, with all shares:', reconstruct_shamir(output_shares))
print('Reconstruction, with 3 shares:', reconstruct_shamir(output_shares[:3]))
print('Reconstruction, with 2 shares:', reconstruct_shamir(output_shares[:2]))

assert reconstruct_shamir(output_shares) == 30
assert reconstruct_shamir(output_shares[:3]) == 30
assert reconstruct_shamir(output_shares[:2]) != 30